In [1]:
#Load all csv
import pandas as pd
import numpy as np
from IPython.display import display

number_of_testers = 33
filenames = ["SL_LIT", "SL_BIG", "FA_LIT", "FA_BIG"]
animname = ["VB", "HS"]
all_dfs = {}

for tester in range(1, number_of_testers+1):
    for session in range(1, 4):
        for trial in range(1, 4):
            for fname in filenames:
                for anim_name in animname:
                    key = f"T{tester}_S{session}_TRY{trial}_{anim_name}_{fname}"
                    path = rf"C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Results_csv\Tester{tester}\Session{session}\Trial{trial}\T{tester}-S{session}-TRY{trial}-{anim_name}_{fname}.csv"
                    try:
                        df = pd.read_csv(path)
                        all_dfs[key] = df
                        print(f"Loaded: {key}")
                    except FileNotFoundError:
                        print(f"Missing: {key}")


Loaded: T1_S1_TRY1_VB_SL_LIT
Loaded: T1_S1_TRY1_HS_SL_LIT
Loaded: T1_S1_TRY1_VB_SL_BIG
Loaded: T1_S1_TRY1_HS_SL_BIG
Loaded: T1_S1_TRY1_VB_FA_LIT
Loaded: T1_S1_TRY1_HS_FA_LIT
Loaded: T1_S1_TRY1_VB_FA_BIG
Loaded: T1_S1_TRY1_HS_FA_BIG
Loaded: T1_S1_TRY2_VB_SL_LIT
Loaded: T1_S1_TRY2_HS_SL_LIT
Loaded: T1_S1_TRY2_VB_SL_BIG
Loaded: T1_S1_TRY2_HS_SL_BIG
Loaded: T1_S1_TRY2_VB_FA_LIT
Loaded: T1_S1_TRY2_HS_FA_LIT
Loaded: T1_S1_TRY2_VB_FA_BIG
Loaded: T1_S1_TRY2_HS_FA_BIG
Loaded: T1_S1_TRY3_VB_SL_LIT
Loaded: T1_S1_TRY3_HS_SL_LIT
Loaded: T1_S1_TRY3_VB_SL_BIG
Loaded: T1_S1_TRY3_HS_SL_BIG
Loaded: T1_S1_TRY3_VB_FA_LIT
Loaded: T1_S1_TRY3_HS_FA_LIT
Loaded: T1_S1_TRY3_VB_FA_BIG
Loaded: T1_S1_TRY3_HS_FA_BIG
Loaded: T1_S2_TRY1_VB_SL_LIT
Loaded: T1_S2_TRY1_HS_SL_LIT
Loaded: T1_S2_TRY1_VB_SL_BIG
Loaded: T1_S2_TRY1_HS_SL_BIG
Loaded: T1_S2_TRY1_VB_FA_LIT
Loaded: T1_S2_TRY1_HS_FA_LIT
Loaded: T1_S2_TRY1_VB_FA_BIG
Loaded: T1_S2_TRY1_HS_FA_BIG
Loaded: T1_S2_TRY2_VB_SL_LIT
Loaded: T1_S2_TRY2_HS_SL_LIT
Loaded: T1_S2_

In [ ]:
#Create the dataframe with the first column, which is the number of fixations
feature_vectors = []
# Loop through all files in the dataframe
for key, df in all_dfs.items():
    #Start from 1 because the fixation with id zero is our first fixation
    num_fix = 1
    if 'FPOGID' in df.columns and not df.empty:
        # Convert to list for easier iteration
        fix_id_series = df['FPOGID'].tolist() 
        # Check that the list is not empty
        if fix_id_series: 
            previous_id = fix_id_series[0]
        # Start from the second element
        for current_id in fix_id_series[1:]:
            if current_id != previous_id:
                num_fix += 1
                previous_id = current_id

    # Create a feature vector with key and features
    feature_vectors.append({
        'file_key': key,
        'f0': num_fix
    })
     
# Create a DataFrame from all feature vectors
feature_df = pd.DataFrame(feature_vectors)

# Display the current dataset
display(feature_df)


,file_key,f0
0,T1_S1_TRY1_VB_SL_LIT,57
1,T1_S1_TRY1_HS_SL_LIT,45
2,T1_S1_TRY1_VB_SL_BIG,59
3,T1_S1_TRY1_HS_SL_BIG,90
4,T1_S1_TRY1_VB_FA_LIT,65
...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42
2324,T33_S3_TRY3_VB_FA_LIT,57
2325,T33_S3_TRY3_HS_FA_LIT,55
2326,T33_S3_TRY3_VB_FA_BIG,58


In [3]:
#Obtain the duration of every FPOGID in a file
import duckdb

fix_duration_vector = []  # Stores the final feature vector per file

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    # SQL query using DuckDB to get max FPOGD per FPOGID
    result = duckdb.query("""
        SELECT FPOGID, MAX(FPOGD) as max_FPOGD
        FROM df
        GROUP BY FPOGID
        ORDER BY FPOGID
    """).to_df()

    # Optional: flatten max_FPOGD column into a single list/vector if needed
    max_fpogd_vector = result['max_FPOGD'].tolist()
    fpogid_vector = result['FPOGID'].tolist()

    # Store in the feature vector list
    fix_duration_vector.append({
        'file_key': key,
        'FPOGID': fpogid_vector,
        'max_FPOGD': max_fpogd_vector
    })

# Create a final DataFrame
fix_duration_vector_df = pd.DataFrame(fix_duration_vector)

# Display the DataFrame
display(fix_duration_vector_df)


,file_key,FPOGID,max_FPOGD
0,T1_S1_TRY1_VB_SL_LIT,"[2139, 2147, 2148, 2149, 2150, 2151, 2152, 215...","[0.04688, 0.29431, 0.31482, 0.11023, 0.1748, 0..."
1,T1_S1_TRY1_HS_SL_LIT,"[2446, 2447, 2448, 2449, 2450, 2451, 2452, 245...","[0.61572, 0.40222, 0.34204, 0.18689, 0.18347, ..."
2,T1_S1_TRY1_VB_SL_BIG,"[2499, 2500, 2501, 2502, 2503, 2504, 2505, 250...","[0.93579, 0.1377, 0.28149, 0.22046, 0.26172, 0..."
3,T1_S1_TRY1_HS_SL_BIG,"[2202, 2207, 2208, 2209, 2210, 2211, 2212, 221...","[0.15417, 0.72949, 0.08789, 0.22131, 0.20044, ..."
4,T1_S1_TRY1_VB_FA_LIT,"[2295, 2305, 2306, 2307, 2308, 2309, 2310, 231...","[0.12073, 1.28003, 0.32092, 0.12061, 0.20105, ..."
...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,"[2168, 2169, 2170, 2171, 2172, 2173, 2174, 217...","[0.46924, 0.5694, 0.17871, 0.15344, 0.15479, 0..."
2324,T33_S3_TRY3_VB_FA_LIT,"[2025, 2028, 2029, 2030, 2031, 2032, 2033, 203...","[0.24872, 0.57495, 0.11774, 0.18781, 0.43475, ..."
2325,T33_S3_TRY3_HS_FA_LIT,"[1964, 1965, 1966, 1967, 1968, 1969, 1970, 197...","[1.9491, 0.26202, 0.20673, 0.32855, 0.28156, 0..."
2326,T33_S3_TRY3_VB_FA_BIG,"[1899, 1900, 1901, 1902, 1903, 1904, 1905, 190...","[0.26117, 0.20367, 0.17419, 0.28778, 0.14081, ..."


In [4]:
#Minimun fixation duration that join with the feature vector
min_fixation_data = []

for item in fix_duration_vector:
    file_key = item['file_key']
    max_fpogd = item['max_FPOGD']

    #Check if empty
    if max_fpogd: 
        min_duration = np.min(max_fpogd)
    else:
        min_duration = None 

    min_fixation_data.append({
        'file_key': file_key,
        'f1': min_duration
    })

# Create pandas DataFrame
min_fixation_df = pd.DataFrame(min_fixation_data)

feature_df = feature_df.merge(min_fixation_df, on='file_key', how='left')
display(feature_df)

#Create the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087
...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448


In [5]:
#Maximum fixation duration that join with the feature vector
max_fixation_data = []

for item in fix_duration_vector:
    file_key = item['file_key']
    max_fpogd = item['max_FPOGD']

    #Check if empty
    if max_fpogd: 
        max_duration = max(max_fpogd)
    else:
        max_duration = None 

    max_fixation_data.append({
        'file_key': file_key,
        'f2': max_duration
    })

# Create pandas DataFrame
max_fixation_df = pd.DataFrame(max_fixation_data)

feature_df = feature_df.merge(max_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003
...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691


In [6]:
#Arithmetic mean of fixation duration that join with feature vector
mean_fixation_data = []

for item in fix_duration_vector:
    file_key = item['file_key']
    max_fpogd = item['max_FPOGD']

    #Check if empty
    if max_fpogd: 
        mean_duration = np.mean(max_fpogd)
    else:
        mean_duration = None 

    mean_fixation_data.append({
        'file_key': file_key,
        'f3': mean_duration
    })

# Create pandas DataFrame
mean_fixation_df = pd.DataFrame(mean_fixation_data)

feature_df = feature_df.merge(mean_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144
...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106


In [7]:
#Geometric mean of fixation duration that join with feature vector
from scipy.stats import gmean

geom_mean_fixation_data = []

for item in fix_duration_vector:
    file_key = item['file_key']
    max_fpogd = item['max_FPOGD']

    #Check if empty
    if max_fpogd: 
        geom_mean_duration = gmean(max_fpogd)
    else:
        geom_mean_duration = None 

    geom_mean_fixation_data.append({
        'file_key': file_key,
        'f4': geom_mean_duration
    })

# Create pandas DataFrame
geom_mean_fixation_df = pd.DataFrame(geom_mean_fixation_data)

feature_df = feature_df.merge(geom_mean_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979
...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703


In [8]:
#Median of fixation duration that join with feature vector
median_fixation_data = []

for item in fix_duration_vector:
    file_key = item['file_key']
    max_fpogd = item['max_FPOGD']

    #Check if empty
    if max_fpogd: 
        median_duration = np.median(max_fpogd)
    else:
        median_duration = None 

    median_fixation_data.append({
        'file_key': file_key,
        'f5': median_duration
    })

# Create pandas DataFrame
median_fixation_df = pd.DataFrame(median_fixation_data)

feature_df = feature_df.merge(median_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190
...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025


In [9]:
#Standard deviation of fixation duration that join with feature vector
std_fixation_data = []

for item in fix_duration_vector:
    file_key = item['file_key']
    max_fpogd = item['max_FPOGD']

    #Check if empty
    if max_fpogd: 
        std_duration = np.std(max_fpogd)
    else:
        std_duration = None 

    std_fixation_data.append({
        'file_key': file_key,
        'f6': std_duration
    })

# Create pandas DataFrame
std_fixation_df = pd.DataFrame(std_fixation_data)

feature_df = feature_df.merge(std_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147
...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850


In [10]:
#Median absolute deviation of fixation duration that join with feature vector
from scipy.stats import median_abs_deviation

mad_fixation_data = []

for item in fix_duration_vector:
    file_key = item['file_key']
    max_fpogd = item['max_FPOGD']

    #Check if empty
    if max_fpogd: 
        mad_duration = median_abs_deviation(max_fpogd, scale=1)
    else:
        mad_duration = None 

    mad_fixation_data.append({
        'file_key': file_key,
        'f7': mad_duration
    })

# Create pandas DataFrame
mad_fixation_df = pd.DataFrame(mad_fixation_data)

feature_df = feature_df.merge(mad_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310
...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200


In [11]:
#Skewness of fixation duration that join with feature vector
from scipy.stats import skew

skew_fixation_data = []

for item in fix_duration_vector:
    file_key = item['file_key']
    max_fpogd = item['max_FPOGD']

    #Check if empty
    if max_fpogd: 
        skew_duration = skew(max_fpogd)
    else:
        skew_duration = None 

    skew_fixation_data.append({
        'file_key': file_key,
        'f8': skew_duration
    })

# Create pandas DataFrame
skew_fixation_df = pd.DataFrame(skew_fixation_data)

feature_df = feature_df.merge(skew_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911
...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149


In [12]:
#Interquartile range of fixation duration that join with feature vector
from scipy.stats import iqr

iqr_fixation_data = []

for item in fix_duration_vector:
    file_key = item['file_key']
    max_fpogd = item['max_FPOGD']

    #Check if empty
    if max_fpogd: 
        iqr_duration = iqr(max_fpogd)
    else:
        iqr_duration = None 

    iqr_fixation_data.append({
        'file_key': file_key,
        'f9': iqr_duration
    })

# Create pandas DataFrame
iqr_fixation_df = pd.DataFrame(iqr_fixation_data)

feature_df = feature_df.merge(iqr_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,0.133180
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,0.261470
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,0.080200
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,0.060668
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,0.134770
...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,0.255223
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,0.127070
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,0.110535
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,0.155365


In [13]:
#Kurtosis of fixation duration that join with feature vector
from scipy.stats import kurtosis

kurt_fixation_data = []

for item in fix_duration_vector:
    file_key = item['file_key']
    max_fpogd = item['max_FPOGD']

    #Check if empty
    if max_fpogd: 
        kurt_duration = kurtosis(max_fpogd)
    else:
        kurt_duration = None 

    kurt_fixation_data.append({
        'file_key': file_key,
        'f10': kurt_duration
    })

# Create pandas DataFrame
kurt_fixation_df = pd.DataFrame(kurt_fixation_data)

feature_df = feature_df.merge(kurt_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,0.133180,0.970100
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,0.261470,-0.020872
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,0.080200,14.786590
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,0.060668,12.008156
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,0.134770,25.635323
...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,0.255223,1.414175
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,0.127070,1.083876
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,0.110535,36.594029
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,0.155365,-0.389986


In [14]:
#FPOGS is the starting time of the fixation POG in seconds since the system initialization or calibration. 
#I used this time to compute the difference between two consecutive fixations in the same file
fpogs_diff_vector = []

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    # Query using DuckDB
    result = duckdb.query("""
        SELECT FPOGID, FPOGS 
        FROM df
        GROUP BY FPOGID, FPOGS
        ORDER BY FPOGID
    """).to_df()

    # Extract vectors
    fpogid_vector = result['FPOGID'].tolist()
    fpogs_vector = result['FPOGS'].tolist()

    # Build list of consecutive FPOGID pairs 
    fpogid_pairs = []
    for i in range(len(fpogid_vector) - 1):
        pair = (fpogid_vector[i], fpogid_vector[i + 1])
        fpogid_pairs.append(pair)

    # Build list of consecutive FPOGS difference 
    fpogs_differences = []
    for i in range(len(fpogs_vector) - 1):
        diff = fpogs_vector[i + 1] - fpogs_vector[i]
        fpogs_differences.append(diff)

    # Append result 
    fpogs_diff_vector.append({
        'file_key': key,
        'FPOGID_pairs': fpogid_pairs,
        'FPOGS_differences': fpogs_differences
    })

# Create the DataFrame
fpogs_diff_vector_df = pd.DataFrame(fpogs_diff_vector)

# Display the DataFrame
display(fpogs_diff_vector_df)


,file_key,FPOGID_pairs,FPOGS_differences
0,T1_S1_TRY1_VB_SL_LIT,"[(2139, 2147), (2147, 2148), (2148, 2149), (21...","[2.793459999999868, 0.3011500000000069, 0.3648..."
1,T1_S1_TRY1_HS_SL_LIT,"[(2446, 2447), (2447, 2448), (2448, 2449), (24...","[0.6225600000000213, 0.4487300000000687, 0.349..."
2,T1_S1_TRY1_VB_SL_BIG,"[(2499, 2500), (2500, 2501), (2501, 2502), (25...","[0.9863299999999526, 0.16406000000006316, 0.30..."
3,T1_S1_TRY1_HS_SL_BIG,"[(2202, 2207), (2207, 2208), (2208, 2209), (22...","[2.69275000000016, 0.7361999999998261, 0.13440..."
4,T1_S1_TRY1_VB_FA_LIT,"[(2295, 2305), (2305, 2306), (2306, 2307), (23...","[1.8618099999998776, 1.2868700000001354, 0.367..."
...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,"[(2168, 2169), (2169, 2170), (2170, 2171), (21...","[0.8433800000000247, 0.5788000000000011, 0.225..."
2324,T33_S3_TRY3_VB_FA_LIT,"[(2025, 2028), (2028, 2029), (2029, 2030), (20...","[0.891110000000026, 0.5984499999999571, 0.1254..."
2325,T33_S3_TRY3_HS_FA_LIT,"[(1964, 1965), (1965, 1966), (1966, 1967), (19...","[1.9954799999999295, 0.2685599999999795, 0.213..."
2326,T33_S3_TRY3_VB_FA_BIG,"[(1899, 1900), (1900, 1901), (1901, 1902), (19...","[0.26886000000001786, 0.24713999999994485, 0.1..."


In [15]:
#Minimum difference between consecutive fixations that join with feature vector
min_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        min_diff = np.min(differences)
    else:
        min_diff = None

    min_diff_data.append({
        'file_key': file_key,
        'f11': min_diff
    })

# Create DataFrame from min differences
min_diff_df = pd.DataFrame(min_diff_data)

feature_df = feature_df.merge(min_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,0.133180,0.970100,0.13403
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,0.261470,-0.020872,0.12097
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,0.080200,14.786590,0.10047
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,0.060668,12.008156,0.08618
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,0.134770,25.635323,0.10034
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,0.255223,1.414175,0.14032
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,0.127070,1.083876,0.09369
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,0.110535,36.594029,0.10083
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,0.155365,-0.389986,0.10266


In [16]:
#Maximum difference between consecutive fixations that join with feature vector
max_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        max_diff = np.max(differences)
    else:
        max_diff = None

    max_diff_data.append({
        'file_key': file_key,
        'f12': max_diff
    })

# Create the DataFrame 
max_diff_df = pd.DataFrame(max_diff_data)

feature_df = feature_df.merge(max_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,0.133180,0.970100,0.13403,2.79346
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,0.261470,-0.020872,0.12097,0.77747
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,0.080200,14.786590,0.10047,0.98633
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,0.060668,12.008156,0.08618,2.69275
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,0.134770,25.635323,0.10034,1.86181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,0.255223,1.414175,0.14032,1.00458
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,0.127070,1.083876,0.09369,0.89111
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,0.110535,36.594029,0.10083,1.99548
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,0.155365,-0.389986,0.10266,0.52337


In [17]:
#Arithmetic mean between consecutive fixations that join with feature vector
mean_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        mean_diff = np.mean(differences)
    else:
        mean_diff = None

    mean_diff_data.append({
        'file_key': file_key,
        'f13': mean_diff
    })

# Create the DataFrame 
mean_diff_df = pd.DataFrame(mean_diff_data)

feature_df = feature_df.merge(mean_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,0.133180,0.970100,0.13403,2.79346,0.317581
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,0.261470,-0.020872,0.12097,0.77747,0.349831
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,0.080200,14.786590,0.10047,0.98633,0.265482
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,0.060668,12.008156,0.08618,2.69275,0.201254
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,0.134770,25.635323,0.10034,1.86181,0.281757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,0.255223,1.414175,0.14032,1.00458,0.396504
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,0.127070,1.083876,0.09369,0.89111,0.287688
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,0.110535,36.594029,0.10083,1.99548,0.306514
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,0.155365,-0.389986,0.10266,0.52337,0.257826


In [18]:
#Geometric mean between consecutive fixations that join with feature vector
gmean_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        gmean_diff = gmean(differences)
    else:
        gmean_diff = None

    gmean_diff_data.append({
        'file_key': file_key,
        'f14': gmean_diff
    })

# Create the DataFrame 
gmean_diff_df = pd.DataFrame(gmean_diff_data)

feature_df = feature_df.merge(gmean_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,0.133180,0.970100,0.13403,2.79346,0.317581,0.264066
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,0.261470,-0.020872,0.12097,0.77747,0.349831,0.308199
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,0.080200,14.786590,0.10047,0.98633,0.265482,0.242950
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,0.060668,12.008156,0.08618,2.69275,0.201254,0.156554
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,0.134770,25.635323,0.10034,1.86181,0.281757,0.238197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,0.255223,1.414175,0.14032,1.00458,0.396504,0.342790
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,0.127070,1.083876,0.09369,0.89111,0.287688,0.261247
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,0.110535,36.594029,0.10083,1.99548,0.306514,0.270139
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,0.155365,-0.389986,0.10266,0.52337,0.257826,0.241596


In [19]:
#Median between consecutive fixations that join with feature vector
median_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        median_diff = np.median(differences)
    else:
        median_diff = None

    median_diff_data.append({
        'file_key': file_key,
        'f15': median_diff
    })

# Create the DataFrame 
median_diff_df = pd.DataFrame(median_diff_data)

feature_df = feature_df.merge(median_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,0.133180,0.970100,0.13403,2.79346,0.317581,0.264066,0.257935
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,0.261470,-0.020872,0.12097,0.77747,0.349831,0.308199,0.294860
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,0.080200,14.786590,0.10047,0.98633,0.265482,0.242950,0.244080
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,0.060668,12.008156,0.08618,2.69275,0.201254,0.156554,0.140630
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,0.134770,25.635323,0.10034,1.86181,0.281757,0.238197,0.241030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,0.255223,1.414175,0.14032,1.00458,0.396504,0.342790,0.311770
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,0.127070,1.083876,0.09369,0.89111,0.287688,0.261247,0.261295
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,0.110535,36.594029,0.10083,1.99548,0.306514,0.270139,0.268255
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,0.155365,-0.389986,0.10266,0.52337,0.257826,0.241596,0.254830


In [20]:
#STD between consecutive fixations that join with feature vector
std_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        std_diff = np.std(differences)
    else:
        std_diff = None

    std_diff_data.append({
        'file_key': file_key,
        'f16': std_diff
    })

# Create the DataFrame 
std_diff_df = pd.DataFrame(std_diff_data)

feature_df = feature_df.merge(std_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,0.133180,0.970100,0.13403,2.79346,0.317581,0.264066,0.257935,0.350696
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,0.261470,-0.020872,0.12097,0.77747,0.349831,0.308199,0.294860,0.179607
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,0.080200,14.786590,0.10047,0.98633,0.265482,0.242950,0.244080,0.138565
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,0.060668,12.008156,0.08618,2.69275,0.201254,0.156554,0.140630,0.287425
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,0.134770,25.635323,0.10034,1.86181,0.281757,0.238197,0.241030,0.252794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,0.255223,1.414175,0.14032,1.00458,0.396504,0.342790,0.311770,0.233169
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,0.127070,1.083876,0.09369,0.89111,0.287688,0.261247,0.261295,0.138470
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,0.110535,36.594029,0.10083,1.99548,0.306514,0.270139,0.268255,0.248476
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,0.155365,-0.389986,0.10266,0.52337,0.257826,0.241596,0.254830,0.089758


In [21]:
#MAD between consecutive fixations that join with feature vector
mad_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        mad_diff = median_abs_deviation(differences, scale=1)
    else:
        mad_diff = None

    mad_diff_data.append({
        'file_key': file_key,
        'f17': mad_diff
    })

# Create the DataFrame 
mad_diff_df = pd.DataFrame(mad_diff_data)

feature_df = feature_df.merge(mad_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,0.133180,0.970100,0.13403,2.79346,0.317581,0.264066,0.257935,0.350696,0.076720
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,0.261470,-0.020872,0.12097,0.77747,0.349831,0.308199,0.294860,0.179607,0.114075
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,0.080200,14.786590,0.10047,0.98633,0.265482,0.242950,0.244080,0.138565,0.044005
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,0.060668,12.008156,0.08618,2.69275,0.201254,0.156554,0.140630,0.287425,0.040160
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,0.134770,25.635323,0.10034,1.86181,0.281757,0.238197,0.241030,0.252794,0.063660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,0.255223,1.414175,0.14032,1.00458,0.396504,0.342790,0.311770,0.233169,0.097170
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,0.127070,1.083876,0.09369,0.89111,0.287688,0.261247,0.261295,0.138470,0.060455
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,0.110535,36.594029,0.10083,1.99548,0.306514,0.270139,0.268255,0.248476,0.059810
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,0.155365,-0.389986,0.10266,0.52337,0.257826,0.241596,0.254830,0.089758,0.073190


In [22]:
#Skewness between consecutive fixations that join with feature vector
skew_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        skew_diff = skew(differences)
    else:
        skew_diff = None

    skew_diff_data.append({
        'file_key': file_key,
        'f18': skew_diff
    })

# Create the DataFrame 
skew_diff_df = pd.DataFrame(skew_diff_data)

feature_df = feature_df.merge(skew_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,0.133180,0.970100,0.13403,2.79346,0.317581,0.264066,0.257935,0.350696,0.076720,6.272821
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,0.261470,-0.020872,0.12097,0.77747,0.349831,0.308199,0.294860,0.179607,0.114075,0.867020
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,0.080200,14.786590,0.10047,0.98633,0.265482,0.242950,0.244080,0.138565,0.044005,3.176352
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,0.060668,12.008156,0.08618,2.69275,0.201254,0.156554,0.140630,0.287425,0.040160,7.426541
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,0.134770,25.635323,0.10034,1.86181,0.281757,0.238197,0.241030,0.252794,0.063660,4.762304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,0.255223,1.414175,0.14032,1.00458,0.396504,0.342790,0.311770,0.233169,0.097170,1.325127
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,0.127070,1.083876,0.09369,0.89111,0.287688,0.261247,0.261295,0.138470,0.060455,1.884942
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,0.110535,36.594029,0.10083,1.99548,0.306514,0.270139,0.268255,0.248476,0.059810,5.793295
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,0.155365,-0.389986,0.10266,0.52337,0.257826,0.241596,0.254830,0.089758,0.073190,0.359743


In [23]:
#Interquartile range between consecutive fixations that join with feature vector
iqr_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        iqr_diff = iqr(differences)
    else:
        iqr_diff = None

    iqr_diff_data.append({
        'file_key': file_key,
        'f19': iqr_diff
    })

# Create the DataFrame 
iqr_diff_df = pd.DataFrame(iqr_diff_data)

feature_df = feature_df.merge(iqr_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,0.970100,0.13403,2.79346,0.317581,0.264066,0.257935,0.350696,0.076720,6.272821,0.150022
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,-0.020872,0.12097,0.77747,0.349831,0.308199,0.294860,0.179607,0.114075,0.867020,0.266113
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,14.786590,0.10047,0.98633,0.265482,0.242950,0.244080,0.138565,0.044005,3.176352,0.086180
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,12.008156,0.08618,2.69275,0.201254,0.156554,0.140630,0.287425,0.040160,7.426541,0.081420
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,25.635323,0.10034,1.86181,0.281757,0.238197,0.241030,0.252794,0.063660,4.762304,0.123387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,1.414175,0.14032,1.00458,0.396504,0.342790,0.311770,0.233169,0.097170,1.325127,0.241760
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,1.083876,0.09369,0.89111,0.287688,0.261247,0.261295,0.138470,0.060455,1.884942,0.116243
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,36.594029,0.10083,1.99548,0.306514,0.270139,0.268255,0.248476,0.059810,5.793295,0.115230
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,-0.389986,0.10266,0.52337,0.257826,0.241596,0.254830,0.089758,0.073190,0.359743,0.139530


In [24]:
#Kurtosis between consecutive fixations that join with feature vector
kurt_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        kurt_diff = kurtosis(differences)
    else:
        kurt_diff = None

    kurt_diff_data.append({
        'file_key': file_key,
        'f20': kurt_diff
    })

# Create the DataFrame 
kurt_diff_df = pd.DataFrame(kurt_diff_data)

feature_df = feature_df.merge(kurt_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,0.13403,2.79346,0.317581,0.264066,0.257935,0.350696,0.076720,6.272821,0.150022,41.389720
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.12097,0.77747,0.349831,0.308199,0.294860,0.179607,0.114075,0.867020,0.266113,-0.160493
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.10047,0.98633,0.265482,0.242950,0.244080,0.138565,0.044005,3.176352,0.086180,12.437481
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.08618,2.69275,0.201254,0.156554,0.140630,0.287425,0.040160,7.426541,0.081420,60.650928
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,0.10034,1.86181,0.281757,0.238197,0.241030,0.252794,0.063660,4.762304,0.123387,24.804236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.14032,1.00458,0.396504,0.342790,0.311770,0.233169,0.097170,1.325127,0.241760,0.819504
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,0.09369,0.89111,0.287688,0.261247,0.261295,0.138470,0.060455,1.884942,0.116243,5.161301
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.10083,1.99548,0.306514,0.270139,0.268255,0.248476,0.059810,5.793295,0.115230,36.583354
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.10266,0.52337,0.257826,0.241596,0.254830,0.089758,0.073190,0.359743,0.139530,-0.192774


In [25]:
#List of all valid(LPV = 1) left pupil diameter(LPD) for each file
lpd_vector = []

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    # Query using DuckDB
    result = duckdb.query("""
        SELECT LPD 
        FROM df
        WHERE LPV = '1'
    """).to_df()

    # Extract vectors
    lpd_list = result['LPD'].tolist()

    # Append result 
    lpd_vector.append({
        'file_key': key,
        'LPD': lpd_list
    })

# Create the DataFrame
lpd_vector_df = pd.DataFrame(lpd_vector)

# Display the DataFrame
display(lpd_vector_df)


,file_key,LPD
0,T1_S1_TRY1_VB_SL_LIT,"[18.33092, 18.23724, 20.23254, 20.18498, 20.33..."
1,T1_S1_TRY1_HS_SL_LIT,"[20.73535, 20.51113, 20.60152, 20.485, 20.4977..."
2,T1_S1_TRY1_VB_SL_BIG,"[17.84583, 18.04667, 18.0429, 17.9603, 17.9667..."
3,T1_S1_TRY1_HS_SL_BIG,"[16.78424, 18.47666, 18.58866, 18.46712, 18.70..."
4,T1_S1_TRY1_VB_FA_LIT,"[20.1371, 20.03338, 19.02217, 19.03521, 19.026..."
...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,"[30.57337, 30.61803, 30.24858, 30.2585, 30.285..."
2324,T33_S3_TRY3_VB_FA_LIT,"[31.52806, 30.68255, 30.68035, 30.68035, 30.76..."
2325,T33_S3_TRY3_HS_FA_LIT,"[28.62523, 28.5755, 31.57859, 31.64881, 31.629..."
2326,T33_S3_TRY3_VB_FA_BIG,"[27.92934, 27.92934, 27.90902, 27.9475, 28.061..."


In [26]:
#Minimum left pupil diameter
min_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        min_lpd = np.min(lpd)
    else:
        min_lpd = None

    min_lpd_data.append({
        'file_key': file_key,
        'f32': min_lpd
    })

# Create DataFrame from min differences
min_lpd_df = pd.DataFrame(min_lpd_data)

feature_df = feature_df.merge(min_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f12,f13,f14,f15,f16,f17,f18,f19,f20,f32
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,2.79346,0.317581,0.264066,0.257935,0.350696,0.076720,6.272821,0.150022,41.389720,14.97845
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.77747,0.349831,0.308199,0.294860,0.179607,0.114075,0.867020,0.266113,-0.160493,9.10561
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.98633,0.265482,0.242950,0.244080,0.138565,0.044005,3.176352,0.086180,12.437481,14.20694
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,2.69275,0.201254,0.156554,0.140630,0.287425,0.040160,7.426541,0.081420,60.650928,16.78424
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,1.86181,0.281757,0.238197,0.241030,0.252794,0.063660,4.762304,0.123387,24.804236,16.44751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,1.00458,0.396504,0.342790,0.311770,0.233169,0.097170,1.325127,0.241760,0.819504,25.92760
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,0.89111,0.287688,0.261247,0.261295,0.138470,0.060455,1.884942,0.116243,5.161301,24.34467
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,1.99548,0.306514,0.270139,0.268255,0.248476,0.059810,5.793295,0.115230,36.583354,20.47150
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.52337,0.257826,0.241596,0.254830,0.089758,0.073190,0.359743,0.139530,-0.192774,25.33441


In [27]:
#Maximum left pupil diameter
max_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        max_lpd = np.max(lpd)
    else:
        max_lpd = None

    max_lpd_data.append({
        'file_key': file_key,
        'f33': max_lpd
    })

# Create DataFrame 
max_lpd_df = pd.DataFrame(max_lpd_data)

feature_df = feature_df.merge(max_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f13,f14,f15,f16,f17,f18,f19,f20,f32,f33
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,0.317581,0.264066,0.257935,0.350696,0.076720,6.272821,0.150022,41.389720,14.97845,21.82685
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.349831,0.308199,0.294860,0.179607,0.114075,0.867020,0.266113,-0.160493,9.10561,23.06998
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.265482,0.242950,0.244080,0.138565,0.044005,3.176352,0.086180,12.437481,14.20694,22.62979
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.201254,0.156554,0.140630,0.287425,0.040160,7.426541,0.081420,60.650928,16.78424,22.31914
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,0.281757,0.238197,0.241030,0.252794,0.063660,4.762304,0.123387,24.804236,16.44751,21.14130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.396504,0.342790,0.311770,0.233169,0.097170,1.325127,0.241760,0.819504,25.92760,31.15751
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,0.287688,0.261247,0.261295,0.138470,0.060455,1.884942,0.116243,5.161301,24.34467,32.63464
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.306514,0.270139,0.268255,0.248476,0.059810,5.793295,0.115230,36.583354,20.47150,32.86868
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.257826,0.241596,0.254830,0.089758,0.073190,0.359743,0.139530,-0.192774,25.33441,30.72208


In [28]:
#Arithmetic mean of left pupil diameter
mean_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        mean_lpd = np.mean(lpd)
    else:
        mean_lpd = None

    mean_lpd_data.append({
        'file_key': file_key,
        'f34': mean_lpd
    })

# Create DataFrame 
mean_lpd_df = pd.DataFrame(mean_lpd_data)

feature_df = feature_df.merge(mean_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f14,f15,f16,f17,f18,f19,f20,f32,f33,f34
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,0.264066,0.257935,0.350696,0.076720,6.272821,0.150022,41.389720,14.97845,21.82685,18.883407
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.308199,0.294860,0.179607,0.114075,0.867020,0.266113,-0.160493,9.10561,23.06998,21.053556
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.242950,0.244080,0.138565,0.044005,3.176352,0.086180,12.437481,14.20694,22.62979,19.978792
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.156554,0.140630,0.287425,0.040160,7.426541,0.081420,60.650928,16.78424,22.31914,20.575229
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,0.238197,0.241030,0.252794,0.063660,4.762304,0.123387,24.804236,16.44751,21.14130,18.973598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.342790,0.311770,0.233169,0.097170,1.325127,0.241760,0.819504,25.92760,31.15751,28.092948
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,0.261247,0.261295,0.138470,0.060455,1.884942,0.116243,5.161301,24.34467,32.63464,28.787894
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.270139,0.268255,0.248476,0.059810,5.793295,0.115230,36.583354,20.47150,32.86868,30.515380
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.241596,0.254830,0.089758,0.073190,0.359743,0.139530,-0.192774,25.33441,30.72208,28.016040


In [29]:
#Geometric mean of left pupil diameter
gmean_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        gmean_lpd = gmean(lpd)
    else:
        gmean_lpd = None

    gmean_lpd_data.append({
        'file_key': file_key,
        'f35': gmean_lpd
    })

# Create DataFrame 
gmean_lpd_df = pd.DataFrame(gmean_lpd_data)

feature_df = feature_df.merge(gmean_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f15,f16,f17,f18,f19,f20,f32,f33,f34,f35
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,0.257935,0.350696,0.076720,6.272821,0.150022,41.389720,14.97845,21.82685,18.883407,18.793943
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.294860,0.179607,0.114075,0.867020,0.266113,-0.160493,9.10561,23.06998,21.053556,21.035456
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.244080,0.138565,0.044005,3.176352,0.086180,12.437481,14.20694,22.62979,19.978792,19.927327
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.140630,0.287425,0.040160,7.426541,0.081420,60.650928,16.78424,22.31914,20.575229,20.561464
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,0.241030,0.252794,0.063660,4.762304,0.123387,24.804236,16.44751,21.14130,18.973598,18.944220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.311770,0.233169,0.097170,1.325127,0.241760,0.819504,25.92760,31.15751,28.092948,28.077206
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,0.261295,0.138470,0.060455,1.884942,0.116243,5.161301,24.34467,32.63464,28.787894,28.732491
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.268255,0.248476,0.059810,5.793295,0.115230,36.583354,20.47150,32.86868,30.515380,30.507258
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.254830,0.089758,0.073190,0.359743,0.139530,-0.192774,25.33441,30.72208,28.016040,27.999061


In [30]:
#Median of left pupil diameter
median_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        median_lpd = np.median(lpd)
    else:
        median_lpd = None

    median_lpd_data.append({
        'file_key': file_key,
        'f36': median_lpd
    })

# Create DataFrame 
median_lpd_df = pd.DataFrame(median_lpd_data)

feature_df = feature_df.merge(median_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f16,f17,f18,f19,f20,f32,f33,f34,f35,f36
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,0.350696,0.076720,6.272821,0.150022,41.389720,14.97845,21.82685,18.883407,18.793943,18.824400
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.179607,0.114075,0.867020,0.266113,-0.160493,9.10561,23.06998,21.053556,21.035456,21.097650
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.138565,0.044005,3.176352,0.086180,12.437481,14.20694,22.62979,19.978792,19.927327,20.053690
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.287425,0.040160,7.426541,0.081420,60.650928,16.78424,22.31914,20.575229,20.561464,20.493495
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,0.252794,0.063660,4.762304,0.123387,24.804236,16.44751,21.14130,18.973598,18.944220,19.005680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.233169,0.097170,1.325127,0.241760,0.819504,25.92760,31.15751,28.092948,28.077206,28.341340
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,0.138470,0.060455,1.884942,0.116243,5.161301,24.34467,32.63464,28.787894,28.732491,28.803940
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.248476,0.059810,5.793295,0.115230,36.583354,20.47150,32.86868,30.515380,30.507258,30.561960
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.089758,0.073190,0.359743,0.139530,-0.192774,25.33441,30.72208,28.016040,27.999061,28.173940


In [32]:
#STD of left pupil diameter
std_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        std_lpd = np.std(lpd)
    else:
        median_lpd = None

    std_lpd_data.append({
        'file_key': file_key,
        'f37': std_lpd
    })

# Create DataFrame 
std_lpd_df = pd.DataFrame(std_lpd_data)

feature_df = feature_df.merge(std_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f18,f19,f20,f32,f33,f34,f35,f36_x,f36_y,f37
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,6.272821,0.150022,41.389720,14.97845,21.82685,18.883407,18.793943,18.824400,1.821947,1.821947
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.867020,0.266113,-0.160493,9.10561,23.06998,21.053556,21.035456,21.097650,0.835551,0.835551
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,3.176352,0.086180,12.437481,14.20694,22.62979,19.978792,19.927327,20.053690,1.418854,1.418854
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,7.426541,0.081420,60.650928,16.78424,22.31914,20.575229,20.561464,20.493495,0.750254,0.750254
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,4.762304,0.123387,24.804236,16.44751,21.14130,18.973598,18.944220,19.005680,1.045301,1.045301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,1.325127,0.241760,0.819504,25.92760,31.15751,28.092948,28.077206,28.341340,0.936039,0.936039
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,1.884942,0.116243,5.161301,24.34467,32.63464,28.787894,28.732491,28.803940,1.771648,1.771648
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,5.793295,0.115230,36.583354,20.47150,32.86868,30.515380,30.507258,30.561960,0.688562,0.688562
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.359743,0.139530,-0.192774,25.33441,30.72208,28.016040,27.999061,28.173940,0.971654,0.971654


In [33]:
#MAD of left pupil diameter
mad_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        mad_lpd = median_abs_deviation(lpd, scale=1)
    else:
        median_lpd = None

    mad_lpd_data.append({
        'file_key': file_key,
        'f38': mad_lpd
    })

# Create DataFrame 
mad_lpd_df = pd.DataFrame(mad_lpd_data)

feature_df = feature_df.merge(mad_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f19,f20,f32,f33,f34,f35,f36_x,f36_y,f37,f38
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,0.150022,41.389720,14.97845,21.82685,18.883407,18.793943,18.824400,1.821947,1.821947,1.773330
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.266113,-0.160493,9.10561,23.06998,21.053556,21.035456,21.097650,0.835551,0.835551,0.373125
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.086180,12.437481,14.20694,22.62979,19.978792,19.927327,20.053690,1.418854,1.418854,0.928220
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.081420,60.650928,16.78424,22.31914,20.575229,20.561464,20.493495,0.750254,0.750254,0.577980
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,0.123387,24.804236,16.44751,21.14130,18.973598,18.944220,19.005680,1.045301,1.045301,0.621870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.241760,0.819504,25.92760,31.15751,28.092948,28.077206,28.341340,0.936039,0.936039,0.574290
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,0.116243,5.161301,24.34467,32.63464,28.787894,28.732491,28.803940,1.771648,1.771648,1.447990
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.115230,36.583354,20.47150,32.86868,30.515380,30.507258,30.561960,0.688562,0.688562,0.389445
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.139530,-0.192774,25.33441,30.72208,28.016040,27.999061,28.173940,0.971654,0.971654,0.703350


In [34]:
#SKEWNESS of left pupil diameter
skew_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        skew_lpd = skew(lpd)
    else:
        median_lpd = None

    skew_lpd_data.append({
        'file_key': file_key,
        'f39': skew_lpd
    })

# Create DataFrame 
skew_lpd_df = pd.DataFrame(skew_lpd_data)

feature_df = feature_df.merge(skew_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f20,f32,f33,f34,f35,f36_x,f36_y,f37,f38,f39
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,41.389720,14.97845,21.82685,18.883407,18.793943,18.824400,1.821947,1.821947,1.773330,-0.188199
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,-0.160493,9.10561,23.06998,21.053556,21.035456,21.097650,0.835551,0.835551,0.373125,-2.127216
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,12.437481,14.20694,22.62979,19.978792,19.927327,20.053690,1.418854,1.418854,0.928220,-0.341561
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,60.650928,16.78424,22.31914,20.575229,20.561464,20.493495,0.750254,0.750254,0.577980,-0.187068
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,24.804236,16.44751,21.14130,18.973598,18.944220,19.005680,1.045301,1.045301,0.621870,-0.460327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.819504,25.92760,31.15751,28.092948,28.077206,28.341340,0.936039,0.936039,0.574290,-0.374585
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,5.161301,24.34467,32.63464,28.787894,28.732491,28.803940,1.771648,1.771648,1.447990,-0.302612
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,36.583354,20.47150,32.86868,30.515380,30.507258,30.561960,0.688562,0.688562,0.389445,-2.365517
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,-0.192774,25.33441,30.72208,28.016040,27.999061,28.173940,0.971654,0.971654,0.703350,-0.278631


In [35]:
#IQR of left pupil diameter
iqr_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        iqr_lpd = iqr(lpd)
    else:
        median_lpd = None

    iqr_lpd_data.append({
        'file_key': file_key,
        'f40': iqr_lpd
    })

# Create DataFrame 
iqr_lpd_df = pd.DataFrame(iqr_lpd_data)

feature_df = feature_df.merge(iqr_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f32,f33,f34,f35,f36_x,f36_y,f37,f38,f39,f40
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,14.97845,21.82685,18.883407,18.793943,18.824400,1.821947,1.821947,1.773330,-0.188199,3.507080
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,9.10561,23.06998,21.053556,21.035456,21.097650,0.835551,0.835551,0.373125,-2.127216,0.751947
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,14.20694,22.62979,19.978792,19.927327,20.053690,1.418854,1.418854,0.928220,-0.341561,1.861880
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,16.78424,22.31914,20.575229,20.561464,20.493495,0.750254,0.750254,0.577980,-0.187068,1.184795
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,16.44751,21.14130,18.973598,18.944220,19.005680,1.045301,1.045301,0.621870,-0.460327,1.245265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,25.92760,31.15751,28.092948,28.077206,28.341340,0.936039,0.936039,0.574290,-0.374585,1.317890
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,24.34467,32.63464,28.787894,28.732491,28.803940,1.771648,1.771648,1.447990,-0.302612,2.953250
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,20.47150,32.86868,30.515380,30.507258,30.561960,0.688562,0.688562,0.389445,-2.365517,0.778917
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,25.33441,30.72208,28.016040,27.999061,28.173940,0.971654,0.971654,0.703350,-0.278631,1.494210


In [36]:
#Kurtosis of left pupil diameter
kurt_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        kurt_lpd = kurtosis(lpd)
    else:
        median_lpd = None

    kurt_lpd_data.append({
        'file_key': file_key,
        'f41': kurt_lpd
    })

# Create DataFrame 
kurt_lpd_df = pd.DataFrame(kurt_lpd_data)

feature_df = feature_df.merge(kurt_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f33,f34,f35,f36_x,f36_y,f37,f38,f39,f40,f41
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,21.82685,18.883407,18.793943,18.824400,1.821947,1.821947,1.773330,-0.188199,3.507080,-1.330764
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,23.06998,21.053556,21.035456,21.097650,0.835551,0.835551,0.373125,-2.127216,0.751947,23.820888
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,22.62979,19.978792,19.927327,20.053690,1.418854,1.418854,0.928220,-0.341561,1.861880,-0.529504
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,22.31914,20.575229,20.561464,20.493495,0.750254,0.750254,0.577980,-0.187068,1.184795,-0.021650
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,21.14130,18.973598,18.944220,19.005680,1.045301,1.045301,0.621870,-0.460327,1.245265,-0.385668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,31.15751,28.092948,28.077206,28.341340,0.936039,0.936039,0.574290,-0.374585,1.317890,-0.399668
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,32.63464,28.787894,28.732491,28.803940,1.771648,1.771648,1.447990,-0.302612,2.953250,-0.581471
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,32.86868,30.515380,30.507258,30.561960,0.688562,0.688562,0.389445,-2.365517,0.778917,27.484172
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,30.72208,28.016040,27.999061,28.173940,0.971654,0.971654,0.703350,-0.278631,1.494210,-0.477196


In [37]:
#List of all valid(RPV = 1) right pupil diameter(RPD) for each file
rpd_vector = []

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    # Query using DuckDB
    result = duckdb.query("""
        SELECT RPD 
        FROM df
        WHERE RPV = '1'
    """).to_df()

    # Extract vectors
    rpd_list = result['RPD'].tolist()

    # Append result 
    rpd_vector.append({
        'file_key': key,
        'RPD': rpd_list
    })

# Create the DataFrame
rpd_vector_df = pd.DataFrame(rpd_vector)

# Display the DataFrame
display(rpd_vector_df)


,file_key,RPD
0,T1_S1_TRY1_VB_SL_LIT,"[17.2877, 17.82784, 18.85434, 18.85434, 18.854..."
1,T1_S1_TRY1_HS_SL_LIT,"[19.8148, 19.63718, 19.91414, 19.88341, 20.081..."
2,T1_S1_TRY1_VB_SL_BIG,"[16.83182, 17.0567, 16.94397, 16.91181, 16.819..."
3,T1_S1_TRY1_HS_SL_BIG,"[16.64259, 18.41689, 18.45479, 18.44613, 18.48..."
4,T1_S1_TRY1_VB_FA_LIT,"[19.08659, 18.89192, 18.82117, 18.85699, 18.70..."
...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,"[29.19351, 29.10667, 28.34218, 28.2825, 28.261..."
2324,T33_S3_TRY3_VB_FA_LIT,"[28.86482, 27.62969, 27.63639, 27.63096, 27.64..."
2325,T33_S3_TRY3_HS_FA_LIT,"[26.51647, 26.5827, 29.29068, 29.39616, 29.267..."
2326,T33_S3_TRY3_VB_FA_BIG,"[27.11602, 27.07738, 27.06608, 27.06064, 27.07..."


In [39]:
#Minimum right pupil diameter
min_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        min_rpd = np.min(rpd)
    else:
        min_rpd = None

    min_rpd_data.append({
        'file_key': file_key,
        'f42': min_rpd
    })

# Create DataFrame from min differences
min_rpd_df = pd.DataFrame(min_rpd_data)

feature_df = feature_df.merge(min_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f35,f36_x,f36_y,f37,f38,f39,f40,f41,f42_x,f42_y
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,18.793943,18.824400,1.821947,1.821947,1.773330,-0.188199,3.507080,-1.330764,25.34366,11.73673
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,21.035456,21.097650,0.835551,0.835551,0.373125,-2.127216,0.751947,23.820888,25.34366,11.33286
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,19.927327,20.053690,1.418854,1.418854,0.928220,-0.341561,1.861880,-0.529504,25.34366,10.67522
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,20.561464,20.493495,0.750254,0.750254,0.577980,-0.187068,1.184795,-0.021650,25.34366,12.66637
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,18.944220,19.005680,1.045301,1.045301,0.621870,-0.460327,1.245265,-0.385668,25.34366,16.05203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,28.077206,28.341340,0.936039,0.936039,0.574290,-0.374585,1.317890,-0.399668,25.34366,23.58247
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,28.732491,28.803940,1.771648,1.771648,1.447990,-0.302612,2.953250,-0.581471,25.34366,22.72544
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,30.507258,30.561960,0.688562,0.688562,0.389445,-2.365517,0.778917,27.484172,25.34366,15.74079
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,27.999061,28.173940,0.971654,0.971654,0.703350,-0.278631,1.494210,-0.477196,25.34366,22.30473


In [40]:
#Maximum right pupil diameter
max_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        max_rpd = np.max(rpd)
    else:
        max_rpd = None

    max_rpd_data.append({
        'file_key': file_key,
        'f43': max_rpd
    })

# Create DataFrame 
max_rpd_df = pd.DataFrame(max_rpd_data)

feature_df = feature_df.merge(max_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f36_x,f36_y,f37,f38,f39,f40,f41,f42_x,f42_y,f43
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,18.824400,1.821947,1.821947,1.773330,-0.188199,3.507080,-1.330764,25.34366,11.73673,20.96601
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,21.097650,0.835551,0.835551,0.373125,-2.127216,0.751947,23.820888,25.34366,11.33286,21.62618
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,20.053690,1.418854,1.418854,0.928220,-0.341561,1.861880,-0.529504,25.34366,10.67522,21.29832
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,20.493495,0.750254,0.750254,0.577980,-0.187068,1.184795,-0.021650,25.34366,12.66637,21.50403
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,19.005680,1.045301,1.045301,0.621870,-0.460327,1.245265,-0.385668,25.34366,16.05203,20.68424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,28.341340,0.936039,0.936039,0.574290,-0.374585,1.317890,-0.399668,25.34366,23.58247,29.19351
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,28.803940,1.771648,1.771648,1.447990,-0.302612,2.953250,-0.581471,25.34366,22.72544,29.06525
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,30.561960,0.688562,0.688562,0.389445,-2.365517,0.778917,27.484172,25.34366,15.74079,30.78447
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,28.173940,0.971654,0.971654,0.703350,-0.278631,1.494210,-0.477196,25.34366,22.30473,28.92718


In [41]:
#Arithmetic mean of right pupil diameter
mean_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        mean_rpd = np.mean(rpd)
    else:
        mean_rpd = None

    mean_rpd_data.append({
        'file_key': file_key,
        'f44': mean_rpd
    })

# Create DataFrame 
mean_rpd_df = pd.DataFrame(mean_rpd_data)

feature_df = feature_df.merge(mean_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f36_y,f37,f38,f39,f40,f41,f42_x,f42_y,f43,f44
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,1.821947,1.821947,1.773330,-0.188199,3.507080,-1.330764,25.34366,11.73673,20.96601,18.064635
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.835551,0.835551,0.373125,-2.127216,0.751947,23.820888,25.34366,11.33286,21.62618,19.355156
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,1.418854,1.418854,0.928220,-0.341561,1.861880,-0.529504,25.34366,10.67522,21.29832,18.887727
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.750254,0.750254,0.577980,-0.187068,1.184795,-0.021650,25.34366,12.66637,21.50403,18.846288
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,1.045301,1.045301,0.621870,-0.460327,1.245265,-0.385668,25.34366,16.05203,20.68424,18.653069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.936039,0.936039,0.574290,-0.374585,1.317890,-0.399668,25.34366,23.58247,29.19351,26.197818
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,1.771648,1.771648,1.447990,-0.302612,2.953250,-0.581471,25.34366,22.72544,29.06525,26.071566
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.688562,0.688562,0.389445,-2.365517,0.778917,27.484172,25.34366,15.74079,30.78447,28.055571
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.971654,0.971654,0.703350,-0.278631,1.494210,-0.477196,25.34366,22.30473,28.92718,25.675444


In [42]:
#Geometric mean of right pupil diameter
gmean_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        gmean_rpd = gmean(rpd)
    else:
        gmean_rpd = None

    gmean_rpd_data.append({
        'file_key': file_key,
        'f45': gmean_rpd
    })

# Create DataFrame 
gmean_rpd_df = pd.DataFrame(gmean_rpd_data)

feature_df = feature_df.merge(gmean_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f37,f38,f39,f40,f41,f42_x,f42_y,f43,f44,f45
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,1.821947,1.773330,-0.188199,3.507080,-1.330764,25.34366,11.73673,20.96601,18.064635,17.999585
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.835551,0.373125,-2.127216,0.751947,23.820888,25.34366,11.33286,21.62618,19.355156,19.328542
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,1.418854,0.928220,-0.341561,1.861880,-0.529504,25.34366,10.67522,21.29832,18.887727,18.844825
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.750254,0.577980,-0.187068,1.184795,-0.021650,25.34366,12.66637,21.50403,18.846288,18.807152
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,1.045301,0.621870,-0.460327,1.245265,-0.385668,25.34366,16.05203,20.68424,18.653069,18.622358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.936039,0.574290,-0.374585,1.317890,-0.399668,25.34366,23.58247,29.19351,26.197818,26.187200
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,1.771648,1.447990,-0.302612,2.953250,-0.581471,25.34366,22.72544,29.06525,26.071566,26.021556
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.688562,0.389445,-2.365517,0.778917,27.484172,25.34366,15.74079,30.78447,28.055571,28.044514
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.971654,0.703350,-0.278631,1.494210,-0.477196,25.34366,22.30473,28.92718,25.675444,25.645190


In [43]:
#Median of right pupil diameter
median_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        median_rpd = np.median(rpd)
    else:
        median_rpd = None

    median_rpd_data.append({
        'file_key': file_key,
        'f46': median_rpd
    })

# Create DataFrame 
median_rpd_df = pd.DataFrame(median_rpd_data)

feature_df = feature_df.merge(median_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f38,f39,f40,f41,f42_x,f42_y,f43,f44,f45,f46
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,1.773330,-0.188199,3.507080,-1.330764,25.34366,11.73673,20.96601,18.064635,17.999585,18.060460
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.373125,-2.127216,0.751947,23.820888,25.34366,11.33286,21.62618,19.355156,19.328542,19.203055
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.928220,-0.341561,1.861880,-0.529504,25.34366,10.67522,21.29832,18.887727,18.844825,19.103850
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.577980,-0.187068,1.184795,-0.021650,25.34366,12.66637,21.50403,18.846288,18.807152,18.396135
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,0.621870,-0.460327,1.245265,-0.385668,25.34366,16.05203,20.68424,18.653069,18.622358,18.833000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.574290,-0.374585,1.317890,-0.399668,25.34366,23.58247,29.19351,26.197818,26.187200,26.345650
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,1.447990,-0.302612,2.953250,-0.581471,25.34366,22.72544,29.06525,26.071566,26.021556,26.163730
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.389445,-2.365517,0.778917,27.484172,25.34366,15.74079,30.78447,28.055571,28.044514,28.043600
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.703350,-0.278631,1.494210,-0.477196,25.34366,22.30473,28.92718,25.675444,25.645190,25.683990


In [45]:
#STD of right pupil diameter
std_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        std_rpd = np.std(rpd)
    else:
        median_rpd = None

    std_rpd_data.append({
        'file_key': file_key,
        'f47': std_rpd
    })

# Create DataFrame 
std_rpd_df = pd.DataFrame(std_rpd_data)

feature_df = feature_df.merge(std_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f40,f41,f42_x,f42_y,f43,f44,f45,f46,f37_y,f47
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,3.507080,-1.330764,25.34366,11.73673,20.96601,18.064635,17.999585,18.060460,1.527503,1.527503
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.751947,23.820888,25.34366,11.33286,21.62618,19.355156,19.328542,19.203055,1.005368,1.005368
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,1.861880,-0.529504,25.34366,10.67522,21.29832,18.887727,18.844825,19.103850,1.258265,1.258265
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,1.184795,-0.021650,25.34366,12.66637,21.50403,18.846288,18.807152,18.396135,1.226277,1.226277
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,1.245265,-0.385668,25.34366,16.05203,20.68424,18.653069,18.622358,18.833000,1.058681,1.058681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,1.317890,-0.399668,25.34366,23.58247,29.19351,26.197818,26.187200,26.345650,0.741422,0.741422
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,2.953250,-0.581471,25.34366,22.72544,29.06525,26.071566,26.021556,26.163730,1.611523,1.611523
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.778917,27.484172,25.34366,15.74079,30.78447,28.055571,28.044514,28.043600,0.755608,0.755608
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,1.494210,-0.477196,25.34366,22.30473,28.92718,25.675444,25.645190,25.683990,1.243970,1.243970


In [46]:
#MAD of right pupil diameter
mad_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        mad_rpd = median_abs_deviation(rpd, scale=1)
    else:
        median_rpd = None

    mad_rpd_data.append({
        'file_key': file_key,
        'f48': mad_rpd
    })

# Create DataFrame 
mad_rpd_df = pd.DataFrame(mad_rpd_data)

feature_df = feature_df.merge(mad_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f41,f42_x,f42_y,f43,f44,f45,f46,f37_y,f47,f48
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,-1.330764,25.34366,11.73673,20.96601,18.064635,17.999585,18.060460,1.527503,1.527503,1.443570
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,23.820888,25.34366,11.33286,21.62618,19.355156,19.328542,19.203055,1.005368,1.005368,0.669080
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,-0.529504,25.34366,10.67522,21.29832,18.887727,18.844825,19.103850,1.258265,1.258265,0.864570
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,-0.021650,25.34366,12.66637,21.50403,18.846288,18.807152,18.396135,1.226277,1.226277,0.647955
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,-0.385668,25.34366,16.05203,20.68424,18.653069,18.622358,18.833000,1.058681,1.058681,0.690400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,-0.399668,25.34366,23.58247,29.19351,26.197818,26.187200,26.345650,0.741422,0.741422,0.389580
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,-0.581471,25.34366,22.72544,29.06525,26.071566,26.021556,26.163730,1.611523,1.611523,1.450710
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,27.484172,25.34366,15.74079,30.78447,28.055571,28.044514,28.043600,0.755608,0.755608,0.413755
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,-0.477196,25.34366,22.30473,28.92718,25.675444,25.645190,25.683990,1.243970,1.243970,0.713330


In [47]:
#SKEWNESS of right pupil diameter
skew_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        skew_rpd = skew(rpd)
    else:
        median_rpd = None

    skew_rpd_data.append({
        'file_key': file_key,
        'f49': skew_rpd
    })

# Create DataFrame 
skew_rpd_df = pd.DataFrame(skew_rpd_data)

feature_df = feature_df.merge(skew_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f42_x,f42_y,f43,f44,f45,f46,f37_y,f47,f48,f49
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,25.34366,11.73673,20.96601,18.064635,17.999585,18.060460,1.527503,1.527503,1.443570,-0.037453
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,25.34366,11.33286,21.62618,19.355156,19.328542,19.203055,1.005368,1.005368,0.669080,-0.295182
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,25.34366,10.67522,21.29832,18.887727,18.844825,19.103850,1.258265,1.258265,0.864570,-0.398767
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,25.34366,12.66637,21.50403,18.846288,18.807152,18.396135,1.226277,1.226277,0.647955,0.535961
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,25.34366,16.05203,20.68424,18.653069,18.622358,18.833000,1.058681,1.058681,0.690400,-0.505731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,25.34366,23.58247,29.19351,26.197818,26.187200,26.345650,0.741422,0.741422,0.389580,-0.560146
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,25.34366,22.72544,29.06525,26.071566,26.021556,26.163730,1.611523,1.611523,1.450710,-0.037291
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,25.34366,15.74079,30.78447,28.055571,28.044514,28.043600,0.755608,0.755608,0.413755,-3.319845
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,25.34366,22.30473,28.92718,25.675444,25.645190,25.683990,1.243970,1.243970,0.713330,-0.035704


In [49]:
#IQR of right pupil diameter
iqr_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        iqr_rpd = iqr(rpd)
    else:
        median_rpd = None

    iqr_rpd_data.append({
        'file_key': file_key,
        'f50': iqr_rpd
    })

# Create DataFrame 
iqr_rpd_df = pd.DataFrame(iqr_rpd_data)

feature_df = feature_df.merge(iqr_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f42_y,f43,f44,f45,f46,f37_y,f47,f48,f49,f50
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,11.73673,20.96601,18.064635,17.999585,18.060460,1.527503,1.527503,1.443570,-0.037453,2.875732
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,11.33286,21.62618,19.355156,19.328542,19.203055,1.005368,1.005368,0.669080,-0.295182,1.408065
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,10.67522,21.29832,18.887727,18.844825,19.103850,1.258265,1.258265,0.864570,-0.398767,2.183140
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,12.66637,21.50403,18.846288,18.807152,18.396135,1.226277,1.226277,0.647955,0.535961,2.123840
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,16.05203,20.68424,18.653069,18.622358,18.833000,1.058681,1.058681,0.690400,-0.505731,1.534745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,23.58247,29.19351,26.197818,26.187200,26.345650,0.741422,0.741422,0.389580,-0.560146,0.843050
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,22.72544,29.06525,26.071566,26.021556,26.163730,1.611523,1.611523,1.450710,-0.037291,2.886730
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,15.74079,30.78447,28.055571,28.044514,28.043600,0.755608,0.755608,0.413755,-3.319845,0.815325
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,22.30473,28.92718,25.675444,25.645190,25.683990,1.243970,1.243970,0.713330,-0.035704,1.403100


In [50]:
#Kurtosis of right pupil diameter
kurt_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        kurt_rpd = kurtosis(rpd)
    else:
        median_rpd = None

    kurt_rpd_data.append({
        'file_key': file_key,
        'f51': kurt_rpd
    })

# Create DataFrame 
kurt_rpd_df = pd.DataFrame(kurt_rpd_data)

feature_df = feature_df.merge(kurt_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f43,f44,f45,f46,f37_y,f47,f48,f49,f50,f51
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,20.96601,18.064635,17.999585,18.060460,1.527503,1.527503,1.443570,-0.037453,2.875732,-1.121120
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,21.62618,19.355156,19.328542,19.203055,1.005368,1.005368,0.669080,-0.295182,1.408065,2.610415
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,21.29832,18.887727,18.844825,19.103850,1.258265,1.258265,0.864570,-0.398767,2.183140,-0.324120
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,21.50403,18.846288,18.807152,18.396135,1.226277,1.226277,0.647955,0.535961,2.123840,-0.584666
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,20.68424,18.653069,18.622358,18.833000,1.058681,1.058681,0.690400,-0.505731,1.534745,-0.680287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,29.19351,26.197818,26.187200,26.345650,0.741422,0.741422,0.389580,-0.560146,0.843050,1.090687
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,29.06525,26.071566,26.021556,26.163730,1.611523,1.611523,1.450710,-0.037291,2.886730,-1.170000
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,30.78447,28.055571,28.044514,28.043600,0.755608,0.755608,0.413755,-3.319845,0.815325,47.851231
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,28.92718,25.675444,25.645190,25.683990,1.243970,1.243970,0.713330,-0.035704,1.403100,-0.159372
